In [1]:
using LinearAlgebra
using Dates

using Plots
using BenchmarkTools

BLAS.set_num_threads(1)

include("src/io.jl")
include("src/distances.jl")
include("src/optimizer.jl")
include("src/network.jl")
include("src/base.jl")
include("src/pretraining.jl")

# Initialize the parameters
globalParms, MCParms, NNParms, preTrainParms, systemParmsList = parametersInit()

# Initialize the input data
inputs = inputInit(globalParms, NNParms, preTrainParms, systemParmsList)
if globalParms.mode == "training"
    model, opt, refRDFs = inputs
else
    model = inputs
end

Running ML-IMC in the training mode.


(Chain(Dense(20 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false)), AMSGrad(0.005, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), Any[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.005, 1.006, 1.005, 1.006, 1.006, 1.007, 1.007, 1.007, 1.007], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.003, 1.003, 1.003, 1.004, 1.004, 1.004, 1.005, 1.005, 1.004, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.004, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.005, 1.004, 1.006, 1.003, 1.004, 1.004, 1.005, 1.004]])

In [34]:
systemParms = systemParmsList[1]

systemParameters("100CH3OH-CG", "methanol-data/100CH3OH/100CH3OH-CG-200.xtc", "methanol-data/100CH3OH/100CH3OH-CG.pdb", 512, "MET", [32.433, 32.433, 32.433], 34116.256126737, "methanol-data/100CH3OH/100CH3OH-CG.rdf", 300, 0.05, 0.0, 298.15, 0.40339559569659034, 1.5, 0.5)

In [42]:
traj = readXTC(systemParms)
frame = read_step(traj, 3)
coordinates = positions(frame);

In [43]:
lengths(UnitCell(frame))

3-element Vector{Float64}:
 32.398998737335205
 32.398998737335205
 32.398998737335205

In [4]:
distanceMatrix = buildDistanceMatrix(frame);

In [5]:
cutoff = 6.0 # Å
rs = 0.0 # Å 
eta = 0.1 # Å^-2
lambda = 1.0
zeta = 1.0;

In [6]:
distanceVector = distanceMatrix[:, 1];

In [7]:
@btime $distanceMatrix[:, 1];

  373.794 ns (1 allocation: 4.12 KiB)


In [8]:
@btime positions($frame)[:, 3];

  51.177 ns (3 allocations: 160 bytes)


In [9]:
@btime positions($frame)[:, 3] .- positions($frame)[:, 4];

  115.786 ns (7 allocations: 400 bytes)


In [10]:
coordinates = positions(frame);

In [11]:
@btime coordinates = positions($frame);

  35.004 ns (2 allocations: 80 bytes)


In [12]:
@btime $coordinates[:, 5];

  19.046 ns (1 allocation: 80 bytes)


In [13]:
@btime distance($frame, 5, 6);

  15.694 ns (0 allocations: 0 bytes)


In [14]:
function computeCosAngle(coordinates, i, j, k, distance_ij, distance_ik)::Float64
    @assert i != j && i != k && k != j 
    vector_0i = coordinates[:, i]
    vector_ij = coordinates[:, j] .- vector_0i
    vector_ik = coordinates[:, k] .- vector_0i
    cosAngle = dot(vector_ij, vector_ik) / (distance_ij * distance_ik)
    return (cosAngle)
end

computeCosAngle (generic function with 1 method)

In [15]:
@btime computeCosAngle($coordinates, 1, 461, 483, distanceVector[461], distanceVector[483])

  138.521 ns (9 allocations: 480 bytes)


0.7169325422695667

In [16]:
function G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    return (
        (1.0 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2)
                ) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff))
end        

G9 (generic function with 3 methods)

In [17]:
function G3(cosAngle, distance_ij, distance_ik, distance_kj, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    return (
        (1.0 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2 +
                (distance_kj - rshift)^2)) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff) *
        distanceCutoff(distance_kj, cutoff))
end        

G3 (generic function with 3 methods)

In [18]:
@btime length($distanceVector);

  1.498 ns (0 allocations: 0 bytes)


In [19]:
function G9total(i, coordinates, distanceMatrix, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    sum = 0.0
    distanceVector = distanceMatrix[:, i];
    N = length(distanceVector)
    @inbounds for k in eachindex(distanceVector)
        distance_ik = distanceVector[k]
        @inbounds for j in 1:k-1
            distance_ij = distanceVector[j]
            if 0 < distance_ij < cutoff && 0 < distance_ik < cutoff
                cosAngle = computeCosAngle(coordinates, i, j, k, distance_ij, distance_ik)
                sum += G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda, rshift)
            end
        end
    end
    return (2.0^(1.0 - zeta) * sum)
end

G9total (generic function with 3 methods)

In [20]:
@btime G9total(5, $coordinates, $distanceMatrix, cutoff, eta, 1.0, 1.0)

  11.537 μs (333 allocations: 29.95 KiB)


0.6491223630240601

In [21]:
G9total(5, coordinates, distanceMatrix, cutoff, eta, 1.0, 1.0)

0.6491223630240601

I will implement the G3total function later (because an additional distance has to be computed)

In [22]:
@btime updateDistance!($frame, $distanceVector, $1);

  10.822 μs (0 allocations: 0 bytes)


In [23]:
include("src/distances.jl");

In [24]:
@btime buildDistanceMatrix_old($frame);

  6.753 ms (2 allocations: 2.00 MiB)


In [25]:
@btime buildDistanceMatrix($frame);

  5.339 ms (3080 allocations: 12.27 MiB)


In [26]:
distanceMatrix = buildDistanceMatrix(frame)

512×512 Matrix{Float64}:
  0.0       7.09189  15.855    17.4607   …   9.68317  15.981    20.8271
  7.09189   0.0      19.4916   14.7913      14.8578   14.2792   17.9268
 15.855    19.4916    0.0      11.6814      19.1468   16.8733   23.2905
 17.4607   14.7913   11.6814    0.0         14.4029   18.7578   16.7659
 12.0278    5.86375  16.1165   13.7371      18.4128   16.9396   16.4657
  6.82342   7.67516  16.2445   13.3721   …  12.6422   17.2571   20.5049
 13.0989   18.7139   17.984    24.9762      11.0264   17.6202   18.7331
 11.3263   10.5723   11.0429   10.3943      14.228    21.6119   20.0425
 19.6562   18.8416   18.4182   13.9998      13.8353   11.5905    6.22577
 15.2346   17.5955   23.1113   24.493       16.9502   11.3124   10.3767
  9.03241  12.3969   11.9791   16.8481   …  15.5818   15.1848   18.55
 15.0417   17.157    15.4381   15.8653      13.3199   15.7629   12.5644
 15.2687   12.0559    9.79153  15.5865      16.0364   18.6892   24.1867
  ⋮                                     

In [27]:
distanceVector = distanceMatrix[:, 1];

In [28]:
@btime $distanceMatrix[:, $1];

  427.540 ns (1 allocation: 4.12 KiB)


In [29]:
coordinates = positions(frame);
r1 = coordinates[:, 1]

3-element Vector{Float64}:
 18.08000087738037
  4.760000109672546
 27.280001640319824

In [30]:
distanceVector1 = distanceMatrix[:, 1];

In [31]:
distanceVector2 = computeDistanceVector(r1, coordinates, systemParms.box);

In [32]:
lengths(UnitCell(frame))

3-element Vector{Float64}:
 32.72618055343628
 32.72618055343628
 32.72618055343628

In [33]:
systemParms.box

3-element Vector{Float64}:
 32.433
 32.433
 32.433